<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/gui/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ─── SETUP FOR DASHBOARD ───

import os, sys

try:
    # Step 1: Clone the GitHub repository if not already present
    if not os.path.exists("/content/Cloud_Computing"):
        !git clone https://github.com/Method-for-Software-System-Development/Cloud_Computing.git /content/Cloud_Computing

    # Step 2: Change directory to project root
    %cd /content/Cloud_Computing

    # Step 3: Checkout the 'develop' branch
    !git fetch origin -q
    !git checkout develop -q

    # Step 4: Add 'logic' directory to Python path
    sys.path.append("/content/Cloud_Computing/logic")

    # Step 5: Install required Python packages (quietly)
    %pip install -q importnb
    %pip install -q paho-mqtt
    %pip install -q -U gradio
    %pip install -q firebase
    %pip install requests beautifulsoup4
    %pip install -q matplotlib

    # Step 6: Import required notebooks from 'logic'
    from importnb import Notebook
    with Notebook():
        import user_controller as uc
        import mqqt_sim_indoor as indoor
        import mqqt_sim_outdoor as outdoor
        import search_index as search
        import sensors_stats as stats

    print("✅ Setup completed successfully.")

except Exception as e:
    print("❌ Setup failed:", str(e))

/content/Cloud_Computing
✅ Setup completed successfully.


In [ ]:
import gradio as gr
import pytz
import datetime

# Helper function for current timestamp
def get_time():
    """
    Returns the current time in Israel timezone.
    """
    tz = pytz.timezone('Asia/Jerusalem')
    now = datetime.datetime.now(tz)
    return now.strftime("%H:%M:%S - %d/%m/%Y")

# State variable to track login status
is_logged_in = gr.State(False)

# Main Gradio UI
with gr.Blocks(css=r"""
:root {
  --dark-blue: #1D3557;
  --blue: #457B9D;
  --light-blue: #A8DADC;
  --mint: #F1FAEE;
  --red: #E63946;
}

#header {
  background: var(--blue);
  padding: 0.8rem 1.2rem;
  display: flex;
  align-items: center;
  min-height: 100px;
  border-radius: 18px;
}

#right_header_panel {
  display: flex;
  flex-direction: column;
  align-items: flex-end;
  justify-content: center;
  height: 100%;
  padding-bottom: 16px;
}

#clock {
    background: var(--blue) !important;
    width: 100% !important;
    text-align: center !important;
    border: 2px solid transparent !important;
    display: flex !important;
    flex-direction: column !important;
    align-items: center !important;
}
#clock textarea {
    font-size: 2rem !important;
    font-weight: bold !important;
    color: white !important;
    background: transparent !important;
    border: none !important;
    outline: none !important;
    text-align: right !important;
    resize: none !important;
    box-shadow: none !important;
    margin: 0 !important;
    padding: 0 !important;
    height: 40px !important;
}

#left_panel {
  background: var(--light-blue);
  padding: 0.8rem 1.2rem;
  display: flex;
  border-radius: 18px;
}

table {
  margin-left: auto;
  margin-right: auto;
  width: 100%;
  background-color: white;
}
th, td {
  background-color: white;
}

#main_panel {
  background: var(--mint);
  padding: 0.8rem 1.2rem;
  display: flex;
  border-radius: 18px;
}

#boxes-row {
  background: var(--mint) !important;
}

.sensor-box {
    border-radius: 18px !important;
    font-weight: bold !important;
    padding: 32px 0px !important;
    width: 210px !important;
    text-align: center !important;
    margin: 12px 18px 8px 18px !important;
    border: 2px solid transparent !important;
    display: flex !important;
    flex-direction: column !important;
    align-items: center !important;
}
.sensor-box textarea {
    font-size: 2.2rem !important;
    font-weight: bold !important;
    color: black !important;
    background: transparent !important;
    border: none !important;
    outline: none !important;
    text-align: center !important;
    resize: none !important;
    box-shadow: none !important;
    margin: 0 !important;
    padding: 0 !important;
    height: 40px !important;
}
.sensor-box label, .sensor-box span {
    font-size: 1.2rem !important;
    color: black !important;
    font-weight: 700 !important;
    text-align: center !important;
    display: block !important;
    margin-bottom: 8px !important;
}
.generating.svelte-ls20lj {
    border: 2px solid transparent !important;
    box-shadow: none !important;
}
div.svelte-633qhp {
    background: transparent !important;
    border: none !important;
}

#indoor-sensor-humidity     { background: #FFADAD !important; }
#indoor-sensor-temperature  { background: #FFD6A5 !important; }
#indoor-sensor-pressure     { background: #FDFFB6 !important; }
#outdoor-sensor-humidity    { background: #CAFFBF !important; }
#outdoor-sensor-temperature { background: #A0C4FF !important; }
#outdoor-sensor-dlight      { background: #BDB2FF !important; }


""") as demo:

    # --------------- HEADER ----------------
    with gr.Row(elem_id="header", equal_height=True):
        with gr.Column(scale=4):
            gr.HTML("""
            <div>
              <h1 style="margin:0;font-size:2rem;font-weight:800;color:white;">OptiLine</h1>
              <h3 style="margin:0;font-size:1.2rem;opacity:0.8;color:white;">CIM & Robotics Lab - Braude College of Engineering</h3>
            </div>
            """)
        with gr.Column(scale=1, elem_id="right_header_panel"):
            clock_txt = gr.Textbox(label="", value=get_time, every=1, interactive=False, elem_id="clock")

    # --------------- MAIN DISPLAY ZONES ----------------

    with gr.Row(equal_height=True):

      with gr.Column(elem_id="left_panel", scale=1):

        # ─── LEFT PANEL: before login ───
        with gr.Column(visible=True) as left_login_panel:
            gr.Markdown("## Sign In:")
            gr.Markdown("### Welcome to the OptiLine system!")
            gr.Markdown("### Please sign in with your username and password.<br> If you don't have an account, contact the lab administrator for access.")
            username = gr.Textbox(label="Username")
            password = gr.Textbox(label="Password", type="password")
            login_msg  = gr.Markdown("")
            with gr.Row():
                clear_btn = gr.Button("Clear")
                submit_btn = gr.Button("Submit")

        # ─── LEFT PANEL: after login ───
        with gr.Column(visible=False) as left_dashboard_panel:
            welcome_txt = gr.Markdown("")
            toGuide = gr.Button("User Guide")
            logout_btn = gr.Button("Logout")
            gr.Markdown("## Main Menu:")
            toSensors = gr.Button("Sensors Dashboard")
            toSearch = gr.Button("MQTT Search Engine")
            toStats = gr.Button("Statistics")
            toSim = gr.Button("Fault Simulator")
            toDirectory = gr.Button("User Directory")
            gr.Markdown("## Leaderboard:")
            leaderboard_txt = gr.Markdown("")

      with gr.Column(elem_id="main_panel", scale=3):

        # ─── RIGHT PANEL: before login ───
        with gr.Column(visible=True) as right_info_panel:
            gr.Markdown("""
            # About the CIM & Robotics Laboratory

            The CIM & Robotics Laboratory at Braude College of Engineering, established in 1997, serves as a cutting-edge educational hub for industrial engineering and mechanical engineering students. Located in Room D106, the lab features semi-industrial CIM (Computer Integrated Manufacturing) and FMS (Flexible Manufacturing System) platforms, providing students with hands-on experience in advanced manufacturing processes, robotics, automation, vision systems, and programmable logic controllers (PLCs). The facility includes equipment such as EMCO lathes and machining centers with integrated robots, CAD and simulation stations, automated storage and retrieval systems (AS/RS), and quality control stations with vision technology. This environment bridges theoretical knowledge with practical skills and prepares students for the rapidly evolving high-tech industry.

            ---

            # About the Application

            - Real-time monitoring and visualization of sensor data, including automatic detection of anomalies and out-of-range values
            - Advanced statistics and trend analysis for production parameters
            - Search and browse capabilities for historical data using the [MQTT.org](https://mqtt.org) protocol
            - Interactive troubleshooting tools for identifying and resolving production issues
            - Secure login and personalized dashboard for each engineer
            - Daily optimization challenges with gamification elements and a live leaderboard
            - Support for various sensor types and seamless integration with laboratory equipment
            """)

        # ─── RIGHT PANEL: live sensors ───
        with gr.Column(visible=False) as right_sensor_panel:
            gr.Markdown("# Live Indoor Sensors Data:")
            with gr.Row(elem_id="boxes-row"):
                humidity_box_in = gr.Textbox(label="Indoor Humidity", show_label=True, elem_id="indoor-sensor-humidity", elem_classes="sensor-box", interactive=False)
                temp_box_in     = gr.Textbox(label="Indoor Temperature", show_label=True, elem_id="indoor-sensor-temperature", elem_classes="sensor-box", interactive=False)
                pressure_box_in = gr.Textbox(label="Pressure", show_label=True, elem_id="indoor-sensor-pressure", elem_classes="sensor-box", interactive=False)

            gr.Markdown("# Live Outdoor Sensors Data:")
            with gr.Row(elem_id="boxes-row"):
                humidity_box_out = gr.Textbox(label="Outdoor Humidity", show_label=True, elem_id="outdoor-sensor-humidity", elem_classes="sensor-box", interactive=False)
                temp_box_out     = gr.Textbox(label="Outdoor Temperature", show_label=True, elem_id="outdoor-sensor-temperature", elem_classes="sensor-box", interactive=False)
                dlight_box_out   = gr.Textbox(label="DayLight (Illuminance)", show_label=True, elem_id="outdoor-sensor-dlight", elem_classes="sensor-box", interactive=False)

        def card_streamer():
            """
            Generator function that yields live sensor data or simulated data for the dashboard.

            Args:
                mode (str): Data source mode.
                    - "mqtt": Receives live data from physical sensors via MQTT.
                    - "simulation": Receives data from a simulated data stream.
                    Default is "mqtt".

            Yields:
                dict: A dictionary mapping each dashboard textbox to its current sensor value.
            """
            # Get data streams for indoor and outdoor sensors according to the selected mode
            for indoor_data, outdoor_data in zip(indoor.get_live_data_stream(mode="simulation"),
                                                outdoor.get_live_data_stream(mode="simulation")):
                yield {
                    humidity_box_in:  f"{indoor_data.get('Humidity', 'N/A')} %",
                    temp_box_in:      f"{indoor_data.get('Temperature', 'N/A')} °C",
                    pressure_box_in:  f"{indoor_data.get('Pressure', 'N/A')} hPa",
                    humidity_box_out: f"{outdoor_data.get('Humidity', 'N/A')} %",
                    temp_box_out:     f"{outdoor_data.get('Temperature', 'N/A')} °C",
                    dlight_box_out:   f"{outdoor_data.get('Dlight', 'N/A')} Lux"
                }

        demo.load(
            fn=card_streamer,
            outputs=[
                humidity_box_in, temp_box_in, pressure_box_in,
                humidity_box_out, temp_box_out, dlight_box_out
            ]
        )

        # ─── RIGHT PANEL: guide ───
        with gr.Column(visible=False) as guide_panel:
            gr.Markdown("# User Guide")

        # ─── RIGHT PANEL: search ───
        with gr.Column(visible=False) as search_panel:
            gr.Markdown("# mqqt.com Website Search Engine")
            gr.Markdown("Type keywords to search the knowledge base. Results will appear below with clickable links.")
            search_query = gr.Textbox(label="Enter words to search", placeholder="Type your query here...")
            search_btn = gr.Button("Search")
            search_results = gr.HTML(label="Search Results")

        # ─── RIGHT PANEL: statistics ───
        with gr.Column(visible=False) as stats_panel:
            gr.Markdown("# Sensors Data Statistics")
            gr.Markdown(
                """
                View and analyze historical data collected every minute from indoor and outdoor sensors.
                Select a date and hour to explore trends in temperature, humidity, light, and pressure over time.
                Interactive plots help you compare sensor values and spot anomalies.
                """
            )

            # Retrieve date strings and mapping from statistics module functions
            unique_date_strings = stats.get_unique_date_strings()
            date_to_hours_map = stats.get_date_to_hours_map()

            if unique_date_strings:
                # Initialize dropdown values
                initial_selected_date_str = unique_date_strings[0]
                initial_hour_choices = date_to_hours_map.get(initial_selected_date_str, [])
                initial_selected_hour_str = initial_hour_choices[0] if initial_hour_choices else None

                date_dropdown = gr.Dropdown(
                    choices=unique_date_strings,
                    value=initial_selected_date_str,
                    label="Select Date"
                )

                time_dropdown = gr.Dropdown(
                    choices=initial_hour_choices,
                    value=initial_selected_hour_str,
                    label="Select Start Hour for 1 Hour Interval"
                )

                with gr.Row():
                    plot_dlight_out = gr.Plot(label="Outdoor Dlight")
                    plot_temp_out = gr.Plot(label="Outdoor Temperature")
                with gr.Row():
                    plot_humidity_out = gr.Plot(label="Outdoor Humidity")
                    plot_humidity_in = gr.Plot(label="Indoor Humidity")
                with gr.Row():
                    plot_temp_in = gr.Plot(label="Indoor Temperature")
                    plot_pressure_in = gr.Plot(label="Indoor Pressure")
                with gr.Row():
                    plot_distance_in = gr.Plot(label="Indoor Distance")

                plot_outputs_list = [
                    plot_dlight_out, plot_temp_out, plot_humidity_out,
                    plot_humidity_in, plot_temp_in, plot_pressure_in, plot_distance_in
                ]

                # Update hour dropdown and plots when the selected date changes
                def update_hour_dropdown_and_plots(selected_date_str_event):
                    # Always fetch the latest mapping to ensure up-to-date options
                    hours_map = stats.get_date_to_hours_map()
                    new_hour_choices = hours_map.get(selected_date_str_event, [])
                    new_selected_hour = new_hour_choices[0] if new_hour_choices else None

                    plot_updates = stats.generate_plots(selected_date_str_event, new_selected_hour)
                    return [gr.update(choices=new_hour_choices, value=new_selected_hour)] + list(plot_updates if plot_updates else [None]*7)

                date_dropdown.change(
                    fn=update_hour_dropdown_and_plots,
                    inputs=date_dropdown,
                    outputs=[time_dropdown] + plot_outputs_list
                )

                time_dropdown.change(
                    fn=stats.generate_plots,
                    inputs=[date_dropdown, time_dropdown],
                    outputs=plot_outputs_list
                )

                # Initial plot generation on UI load
                demo.load(
                    fn=stats.generate_plots,
                    inputs=[date_dropdown, time_dropdown],
                    outputs=plot_outputs_list
                )
            else:
                gr.Markdown("No data available to determine dates from Firebase.")

        # ─── RIGHT PANEL: directory ───
        with gr.Column(visible=False) as directory_panel:
          gr.Markdown("## User Directory")
          phonebook_txt = gr.Markdown("")
    # --------------- LOGIC ----------------

    def clear_fields():
        return "", ""

    def do_login(user, pw):
        """
        Handles user login. Updates all dashboard panels and user info on successful authentication.

        Args:
            user (str): Username entered in the login form.
            pw (str): Password entered in the login form.

        Returns:
            list: Gradio UI updates for all relevant components.
        """
        if uc.login(user, pw):
            # Retrieve user's profile from Firebase
            _, profile = uc.fb.get_user(user)
            name = profile.get("first_name", user)
            role = profile.get("role", "")
            user_name = profile.get("username", user)

            # Build a personalized welcome message
            welcome_text = f"### Welcome back, **{name}** ({role})"

            # Retrieve leaderboard and user's rank
            leaderboard_list, user_rank = uc.get_leaderboard(user_name)

            # Build leaderboard Markdown table with medals
            leaderboard_md = "#### \n| Rank | Username | Score |\n|:----:|:-----------|:------:|\n"
            for rank, uname, score in leaderboard_list:
                medal = ""
                if rank == 1: medal = "🥇"
                elif rank == 2: medal = "🥈"
                elif rank == 3: medal = "🥉"
                you = " <b>(You)</b>" if uname == user_name else ""
                name_str = f"**{uname}{you}**" if uname == user_name else uname
                score_str = f"**{score}**" if uname == user_name else str(score)
                leaderboard_md += f"| {rank} {medal} | {name_str} | {score_str} |\n"

            # Show dashboard panels after login: left_dashboard_panel, right_sensor_panel
            # Hide left_login_panel, right_info_panel, search_panel, stats_panel
            return [
                gr.update(visible=False),   # left_login_panel
                gr.update(visible=True),    # left_dashboard_panel
                gr.update(visible=False),   # right_info_panel
                gr.update(visible=True),    # right_sensor_panel
                gr.update(value=True),      # is_logged_in
                "",                         # login_msg
                gr.update(value=welcome_text),       # welcome_txt
                gr.update(value=leaderboard_md),     # leaderboard_txt
            ]
        else:
            # Failed login: show error message, keep panels unchanged
            return [
                gr.update(), gr.update(), gr.update(), gr.update(),
                gr.update(value=False), "Incorrect username or password",
                gr.update(), gr.update()
            ]

    def do_logout():
        """
        Handles user logout. Restores the login screen and hides all dashboard panels.

        Returns:
            list: Gradio UI updates for all relevant components.
        """
        return [
            gr.update(visible=True),    # left_login_panel
            gr.update(visible=False),   # left_dashboard_panel
            gr.update(visible=True),    # right_info_panel

            gr.update(visible=False),   # right_sensor_panel
            gr.update(visible=False),   # search_panel
            gr.update(visible=False),   # stats_panel
            gr.update(visible=False),   # guide_panel
            gr.update(visible=False),   # directory_panel

            gr.update(value=False),     # is_logged_in
            "",                         # login_msg
            gr.update(value=""),        # welcome_txt
            gr.update(value=""),        # leaderboard_txt
        ]

    def show_sensor_panel():
        """
        Shows the live sensors panel, hides all other panels.
        """
        return [
            gr.update(visible=True),   # right_sensor_panel
            gr.update(visible=False),  # search_panel
            gr.update(visible=False),  # stats_panel
            gr.update(visible=False),  # guide_panel
            gr.update(visible=False),  # directory_panel
        ]

    def show_search_panel():
        """
        Shows the search panel, hides all other panels.
        """
        return [
            gr.update(visible=False),  # right_sensor_panel
            gr.update(visible=True),   # search_panel
            gr.update(visible=False),  # stats_panel
            gr.update(visible=False),  # guide_panel
            gr.update(visible=False),  # directory_panel
        ]

    def show_stats_panel():
        """
        Shows the statistics panel, hides all other panels.
        """
        return [
            gr.update(visible=False),  # right_sensor_panel
            gr.update(visible=False),  # search_panel
            gr.update(visible=True),   # stats_panel
            gr.update(visible=False),  # guide_panel
            gr.update(visible=False),  # directory_panel
        ]

    def show_guide_panel():
        """
        Shows the usage guide panel, hides all other panels.
        """
        return [
            gr.update(visible=False),  # right_sensor_panel
            gr.update(visible=False),  # search_panel
            gr.update(visible=False),  # stats_panel
            gr.update(visible=True),   # guide_panel
            gr.update(visible=False),  # directory_panel
        ]

    def show_directory_panel():
        """
        Shows the user directory panel, hides all other panels.
        """
        return [
            gr.update(visible=False),  # right_sensor_panel
            gr.update(visible=False),  # search_panel
            gr.update(visible=False),  # stats_panel
            gr.update(visible=False),  # guide_panel
            gr.update(visible=True),   # directory_panel
        ]
    def generate_phonebook_table():
      """
      Generates a Markdown table representing the user directory (phonebook),
      including name, role, and phone number, sorted alphabetically by last name.

      Returns:
          str: A Markdown-formatted string containing the directory table.
      """
      status, phonebook = uc.fb.get_phonebook()

      if status != 200 or not phonebook:
          return "Failed to load phonebook."

      # Sort alphabetically by last name
      phonebook.sort(key=lambda x: x["name"].split()[-1].lower())

      # Build Markdown table: Name (left), Role (middle), Phone (right)
      md = "#### \n| Name | Role | Phone |\n|:----------------|:----------------------|:------------|\n"
      for entry in phonebook:
          name = entry.get("name", "N/A")
          role = entry.get("role", "N/A")
          phone = entry.get("phone", "N/A")
          md += f"| {name} | {role} | {phone} |\n"

      return md


    # Button callbacks
    toSensors.click(fn=show_sensor_panel, outputs=[right_sensor_panel, search_panel, stats_panel, guide_panel, directory_panel])
    toSearch.click(fn=show_search_panel, outputs=[right_sensor_panel, search_panel, stats_panel, guide_panel, directory_panel])
    toStats.click(fn=show_stats_panel, outputs=[right_sensor_panel, search_panel, stats_panel, guide_panel, directory_panel])
    toGuide.click(fn=show_guide_panel, outputs=[right_sensor_panel, search_panel, stats_panel, guide_panel, directory_panel])
    toDirectory.click(
    fn=lambda: [*show_directory_panel(), generate_phonebook_table()],
    outputs=[right_sensor_panel, search_panel, stats_panel, guide_panel, directory_panel, phonebook_txt]
)


    clear_btn.click(fn=clear_fields, outputs=[username, password])

    submit_btn.click(
        fn=do_login,
        inputs=[username, password],
        outputs=[
            left_login_panel,
            left_dashboard_panel,
            right_info_panel,
            right_sensor_panel,
            is_logged_in,
            login_msg,
            welcome_txt,
            leaderboard_txt,
        ]
    )

    logout_btn.click(
        fn=do_logout,
        outputs=[
            left_login_panel,
            left_dashboard_panel,
            right_info_panel,
            right_sensor_panel,
            search_panel,
            stats_panel,
            guide_panel,
            directory_panel,
            is_logged_in,
            login_msg,
            welcome_txt,
            leaderboard_txt,
        ]
    )

    search_btn.click(
        fn=search.search_words,
        inputs=search_query,
        outputs=search_results
    )


# Launch app
demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b2eac166cbf24b71d4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
